The data has been cleaned and stored in pickle files in p1_clean.ipynb.


In [2]:
import logging, importlib, sys, tqdm
import spacy
import pandas as pd
import os, re
import pickle
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from _pckle import save_pickle_object, load_pickle_object
from _logging import set_logging
from _graph import histplot_count, histplot_range_count
from _utility import gl

set_logging(logging)
df_text = load_pickle_object(gl.pkl_df_text)
df_is_business = load_pickle_object(gl.pkl_df_is_business)


2023-01-03 15:06:42,911 | INFO : Loading pickle file from: pickle\pkl_df_text.pkl
2023-01-03 15:06:43,171 | INFO : Loading pickle file from: pickle\pkl_df_is_business.pkl


Find out the balance of the dataset

In [3]:
def get_balance_metric(df_is_business):
    num_rows = len(df_is_business)
    num_business = df_is_business.sum(axis=0)
    number_in_business_category = num_business.values[0]
    perc_business = (number_in_business_category / num_rows) * 100
    return perc_business


In [4]:
perc_business = get_balance_metric(df_is_business)
logging.info(f"The number of tweets in the business category are {round(perc_business, 2)}%")

2023-01-03 15:06:43,260 | INFO : The number of tweets in the business category are 20.94%


Now vectorise the tweets

In [7]:
vectorizer = CountVectorizer(stop_words='english')
X_vec = vectorizer.fit_transform(df_text[gl.text])
X_vec = X_vec.todense()
X_vec 

MemoryError: Unable to allocate 1.09 TiB for an array with shape (785916, 189828) and data type int64

In [ ]:
tfidf = TfidfTransformer() # by default applies "l2" normalization
X_tfidf = tfidf.fit_transform(X_vec)
X_tfidf = X_tfidf.todense()
X_tfidf

In [ ]:
save_pickle_object(gl.pkl_X_tfidf, X_tfidf)